<a href="https://colab.research.google.com/github/sexymonster/Projects/blob/main/TeamProject/TomatoDataCustom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.인트로

## 1-1. 패키지 로드

In [ ]:
from google.colab import drive
import glob,os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
import matplotlib

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# 버전 오류를 잡아주는 코드
!pip install -U pandas-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 268 kB 8.8 MB/s 
     |████████████████████████████████| 102 kB 52.6 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 690 kB 47.1 MB/s 
     |████████████████████████████████| 4.7 MB 38.8 MB/s 
     |████████████████████████████████| 9.8 MB 40.9 MB/s 
     |████████████████████████████████| 296 kB 66.5 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=bfb5e3de66b56b0536a5781a5ac9c3d6423954212679e584e42bebc8e92af792
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
Successfully built htmlmin
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2
  Attempting uninstall: requests
   

## 1-2. 데이터 로드

In [ ]:
path = "/content/gdrive/MyDrive/tomatoA/"

In [ ]:
file_list = os.listdir(path)
print(file_list)

['2019_0010057_01', '2020_0020738_01', '2021_0010071_01', '2021_0020760_01']


## 1-3. 데이터 형태 수정

In [ ]:
# 농가 번호를 입력하면 그 농가의 생육 정보(live), 제어 정보(ctrl), 환경 정보(env)들을 출력해주는 함수
def dataset(file, path=path):
  try:
    live = pd.read_excel(path + file+ "/방울토마토_생육정보_일별.xlsx")
    live = live.rename(columns = live.iloc[0])
    live = live.drop(live.index[0])
  except:
    pass

  ctrl = pd.read_excel(path + file+ "/방울토마토_제어정보_일별.xlsx")
  ctrl = ctrl.rename(columns = ctrl.iloc[0])
  ctrl = ctrl.drop(ctrl.index[0])

  env = pd.read_excel(path + file+ "/방울토마토_환경정보_일별.xlsx")
  env = env.rename(columns = env.iloc[0])
  env = env.drop(env.index[0])
  env = env.drop(env.columns[[3,5,6,9,10,11,13,14,19]], axis = 1)

  try:
    return live, ctrl, env
  except:
    return ctrl,env

In [ ]:
# 모든 농가 데이터에 각각 번호(예를들면 ctrl1, ctrl2처럼)를 붙여줘서 출력해줌
file_list = os.listdir(path)

i = 1
live_list = []
ctrl_list = []
env_list= []

for file in file_list:
  try:
    globals()["live"+str(i)], globals()["ctrl"+str(i)], globals()["env"+str(i)]  = dataset(file,path)
    # live_list.append(globals()["live"+str(i)])
    # ctrl_list.append(globals()["ctrl"+str(i)])
    # env_list.append(globals()["env"+str(i)])
  except:
    globals()["ctrl"+str(i)], globals()["env"+str(i)]  = dataset(file,path)
    # ctrl_list.append(globals()["ctrl"+str(i)])
    # env_list.append(globals()["env"+str(i)])
  i += 1
print(i-1, '개의 농장 데이터가 있습니다.')

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



4 개의 농장 데이터가 있습니다.


# 2. 전처리

## 2-1. 데이터 전처리 함수

adapt(df) : 필요한 컬럼들로만 이루어진 데이터 프레임 생성\
astype(df) : 데이터들의 타입 변경 

### 2-1-1. 필요없는 컬럼 삭제해주는 함수 adapt()

In [ ]:
# 유효한 컬럼을 알려주는 함수
def effcolm(df):
  effective = []
  columns = list(df)

  # 유효한 컬럼 추출 (nan값 개수가 1/3 이하인 컬럼)
  for col in columns:
    num_nan = df[col].isnull().sum()

    if num_nan < len(df[col])/3:
      effective.append(col)
  
  # 전부 0으로 표기된 의미 없는 컬럼도 제거(Nan,0.0이 같이 있으면 2개로 표시됨)
  for col in effective:
    if len(df[col].unique()) < 3:
      effective.remove(col) 


  return effective
  

In [ ]:
# 유효한 컬럼으로만 이루어진 데이터프레임 생성
def adapt(df):
  eff_col = effcolm(df)
  df = df[eff_col]
  
  return df

### 2-1-2. Date를 인덱스로 설정하는 함수 astype()

In [ ]:
# dtype변경 함수

def astype(df):
  # 수집일, 조사일의 데이터 형태를 datetime으로 변경하고, 인덱스로 설정
  df['Date'] = pd.to_datetime(df.iloc[:,0])
  df = df.set_index(keys="Date", drop = True)
  df = df.drop(df.columns[0], axis=1)

  # 나머지 데이터들의 형태를 float으로 변경
  df = df.astype("float")

  return df

### 2-1-3. 생육 데이터 자료의 변화량을 구해주는 함수 delta()

In [ ]:
# 생육 데이터 자료의 변화량을 구해주는 함수
def delta(live):
  live_new = live[["주차","최종화방번호(번)"]]
  live_new["생장길이변화(dcm)"] = live.sort_values("Date")["생장길이(cm)"].pct_change()
  live_new["생장길이변화(dcm)"] = live.sort_values("Date")["화방높이(cm)"].pct_change()
  live_new["줄기굵기변화(dmm)"] = live.sort_values("Date")["줄기굵기(mm)"].pct_change()
  live_new["엽장변화(dcm)"] = live.sort_values("Date")["엽장(cm)"].pct_change()
  live_new["엽폭변화(dcm)"] = live.sort_values("Date")["엽폭(cm)"].pct_change()
  live_new["엽수변화(개)"] = live.sort_values("Date")["엽수(개)"].pct_change()
  live_new["개화군변화(점)"] = live.sort_values("Date")["개화군(점)"].pct_change()
  live_new["착과군변화(점)"] = live.sort_values("Date")["착과군(점)"].pct_change()
  live_new["열매수변화(개)"] = live.sort_values("Date")["열매수(개)"].pct_change()

  return live_new

### 2-1-4. adapt() 와 astype()을 동시에 실행시켜주는 함수 changedata()

In [ ]:
def changedata(df):
  df = adapt(df)
  df = astype(df)
  
  return df

### 2-1-5. 생육데이터를 일 단위로 데이터 증강 시켜주는 함수 augment()

In [ ]:
def augment(df):
  df_idx = df.index.to_list()
  print()
  idx = []
  growlist = []

  df_new = pd.DataFrame(columns=['날짜', '생장길이(cm)', '화방높이(cm)', '줄기굵기(mm)', '엽장(cm)', '엽폭(cm)', '엽수(개)', 
                           '개화군(점)', '착과군(점)', '열매수(개)', '최종화방번호(번)'])
  for i in range(len(df_idx)-1):
    start = df_idx[i]
    start_gr = df.iloc[i,1]
    start_ht = df.iloc[i,2]
    start_th = df.iloc[i,3]
    start_ll = df.iloc[i,4]
    start_lw = df.iloc[i,5]
    start_ln = df.iloc[i,6]
    start_gh = df.iloc[i,7]
    start_ck = df.iloc[i,8]
    start_tm = df.iloc[i,9]
    start_nb = df.iloc[i,10]


    end = df_idx[i+1]
    end_gr = df.iloc[i+1,1]
    end_ht = df.iloc[i+1,2]
    end_th = df.iloc[i+1,3]
    end_ll = df.iloc[i+1,4]
    end_lw = df.iloc[i+1,5]
    end_ln = df.iloc[i+1,6]
    end_gh = df.iloc[i+1,7]
    end_ck = df.iloc[i+1,8]
    end_tm = df.iloc[i+1,9]
    end_nb = df.iloc[i+1,10]

    # print(start,start_gr,end,end_gr)
    diff = end_gr - start_gr
    dates = pd.date_range(start,end)
    inter = len(dates) - 1

    for j in range(inter):
      day = dates[j]
      growth = start_gr + (end_gr - start_gr)*j/(inter)
      height = start_ht + (end_ht - start_ht)*j/(inter)
      thick = start_th + (end_th - start_th)*j/(inter)
      lea_len = start_ll + (end_ll - start_ll)*j/(inter)
      lea_wid = start_lw + (end_lw - start_lw)*j/(inter)
      lea_num = start_ln + (end_ln - start_ln)*j/(inter)
      gaehwa = start_gh + (end_gh - start_gh)*j/(inter)
      chaka = start_ck + (end_ck - start_ck)*j/(inter)
      tomato = start_tm + (end_tm - start_tm)*j/(inter)
      numb = start_nb + (end_nb - start_nb)*j/(inter)
      

      df_new = df_new.append(pd.DataFrame([[day, growth, height, thick, lea_len, lea_wid, lea_num, gaehwa, chaka, tomato, numb]], columns=['날짜', '생장길이(cm)', '화방높이(cm)', '줄기굵기(mm)', '엽장(cm)', '엽폭(cm)', '엽수(개)', 
                            '개화군(점)', '착과군(점)', '열매수(개)', '최종화방번호(번)']), ignore_index=True)
  df_new = astype(df_new)
  df_new = df_new.round(2)
  return df_new


## 2-2. 데이터 처리 과정

### 2-2-1. 환경데이터

In [ ]:
env1 = changedata(env1)
env2 = changedata(env2)
env3 = changedata(env3)
env4 = changedata(env4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
env4.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 197 entries, 2021-07-29 to 2022-03-18
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   내부CO2(ppm)    167 non-null    float64
 1   이슬점온도(도)      171 non-null    float64
 2   토양EC(dS/m)    158 non-null    float64
 3   내부습도(%)       170 non-null    float64
 4   지습(%)         170 non-null    float64
 5   강우감지          158 non-null    float64
 6   일사량(W/m-2·s)  166 non-null    float64
 7   외부온도(도)       167 non-null    float64
 8   내부온도(도)       172 non-null    float64
 9   지온(도)         171 non-null    float64
 10  외부풍속(m/s)     167 non-null    float64
dtypes: float64(11)
memory usage: 18.5 KB


In [ ]:
# for i in range(len(new_live2)):
#   dlist = new_live2.index.to_list

#   for x in range(dlist[i],dlist[i+1]):
#       start = x
#       temp = new_live2[(new_live2["height"] >= start) & (new_live2["height"] < end)]
#       print("{}이상 {}미만 : {}".format(start, end, temp["height"].mean()))


### 2-2-2. 생육데이터

In [ ]:
live1 = changedata(live1)
live2 = changedata(live2)
live3 = changedata(live3)
live4 = changedata(live4)

In [ ]:
live2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17 entries, 2021-01-09 to 2021-04-28
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   주차         17 non-null     float64
 1   생장길이(cm)   17 non-null     float64
 2   화방높이(cm)   17 non-null     float64
 3   줄기굵기(mm)   17 non-null     float64
 4   엽장(cm)     17 non-null     float64
 5   엽폭(cm)     17 non-null     float64
 6   엽수(개)      17 non-null     float64
 7   개화군(점)     17 non-null     float64
 8   착과군(점)     17 non-null     float64
 9   열매수(개)     17 non-null     float64
 10  최종화방번호(번)  17 non-null     float64
dtypes: float64(11)
memory usage: 1.6 KB


In [ ]:
live2

,주차,생장길이(cm),화방높이(cm),줄기굵기(mm),엽장(cm),엽폭(cm),엽수(개),개화군(점),착과군(점),열매수(개),최종화방번호(번)
Date,,,,,,,,,,,
2021-01-09,15.0,0.0,193.8,9.60,363.8,272.5,9.75,4.75,6.75,68.70,8.75
2021-01-13,16.0,13.7,208.0,9.90,377.5,278.0,10.00,4.25,7.00,90.00,8.50
2021-01-20,17.0,13.2,142.3,8.88,361.5,265.5,12.00,4.50,8.25,100.75,9.50
2021-01-27,18.0,22.5,160.0,10.40,340.5,278.5,12.75,4.25,9.00,102.75,10.00
2021-02-03,19.0,12.8,128.3,8.33,337.5,271.3,17.00,4.25,8.75,110.75,9.50
2021-02-10,20.0,7.5,147.5,9.25,396.3,355.0,16.00,4.25,8.75,112.25,9.25
2021-02-17,21.0,14.8,181.3,9.60,403.3,352.5,13.75,4.00,9.00,110.25,9.00
2021-02-24,22.0,18.5,207.3,10.30,312.5,275.0,14.00,3.00,9.00,78.43,9.50
2021-03-03,23.0,10.8,187.8,9.60,317.5,287.5,15.50,3.75,9.00,101.25,10.00


### 2-2-2(1). 생육데이터 증강

In [ ]:
new_live1 = augment(live1)
new_live2 = augment(live2)
new_live3 = augment(live3)
new_live4 = augment(live4)

In [ ]:
new_live1

,생장길이(cm),화방높이(cm),줄기굵기(mm),엽장(cm),엽폭(cm),엽수(개),개화군(점),착과군(점),열매수(개),최종화방번호(번)
Date,,,,,,,,,,
2019-12-07,1807.50,172.50,10.54,382.50,212.50,15.50,5.04,3.91,27.00,5.75
2019-12-08,1567.14,182.86,10.51,377.86,220.00,14.04,5.18,4.06,28.54,5.89
2019-12-09,1326.79,193.21,10.48,373.21,227.50,12.57,5.31,4.21,30.07,6.04
2019-12-10,1086.43,203.57,10.45,368.57,235.00,11.11,5.45,4.36,31.61,6.18
2019-12-11,846.07,213.93,10.43,363.93,242.50,9.64,5.58,4.52,33.14,6.32
...,...,...,...,...,...,...,...,...,...,...
2020-05-25,391.43,307.50,6.08,208.21,131.43,15.36,4.22,3.41,25.43,4.79
2020-05-26,447.14,300.00,6.14,216.07,132.14,15.79,4.34,3.59,26.39,4.93
2020-05-27,502.86,292.50,6.21,223.93,132.86,16.21,4.45,3.77,27.36,5.07


In [ ]:
del_live2 = delta(live2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
# live2_idx = live2.index.to_list()
# idx = []
# growlist = []
# df = pd.DataFrame(columns=['날짜', '생장길이(cm)', '화방높이(cm)', '줄기굵기(mm)', '엽장(cm)', '엽폭(cm)', '엽수(개)', 
#                            '개화군(점)', '착과군(점)', '열매수(개)', '최종화방번호(번)'])
# for i in range(len(live2_idx)-1):
#   start = live2_idx[i]
#   start_gr = live2.iloc[i,1]
#   start_ht = live2.iloc[i,2]
#   start_th = live2.iloc[i,3]
#   start_ll = live2.iloc[i,4]
#   start_lw = live2.iloc[i,5]
#   start_ln = live2.iloc[i,6]
#   start_gh = live2.iloc[i,7]
#   start_ck = live2.iloc[i,8]
#   start_tm = live2.iloc[i,9]
#   start_nb = live2.iloc[i,10]


#   end = live2_idx[i+1]
#   end_gr = live2.iloc[i+1,1]
#   end_ht = live2.iloc[i+1,2]
#   end_th = live2.iloc[i+1,3]
#   end_ll = live2.iloc[i+1,4]
#   end_lw = live2.iloc[i+1,5]
#   end_ln = live2.iloc[i+1,6]
#   end_gh = live2.iloc[i+1,7]
#   end_ck = live2.iloc[i+1,8]
#   end_tm = live2.iloc[i+1,9]
#   end_nb = live2.iloc[i+1,10]

#   # print(start,start_gr,end,end_gr)
#   diff = end_gr - start_gr
#   dates = pd.date_range(start,end)
#   inter = len(dates) - 1

#   for j in range(inter):
#     day = dates[j]
#     growth = start_gr + (end_gr - start_gr)*j/(inter)
#     height = start_ht + (end_ht - start_ht)*j/(inter)
#     thick = start_th + (end_th - start_th)*j/(inter)
#     lea_len = start_ll + (end_ll - start_ll)*j/(inter)
#     lea_wid = start_lw + (end_lw - start_lw)*j/(inter)
#     lea_num = start_ln + (end_ln - start_ln)*j/(inter)
#     gaehwa = start_gh + (end_gh - start_gh)*j/(inter)
#     chaka = start_ck + (end_ck - start_ck)*j/(inter)
#     tomato = start_tm + (end_tm - start_tm)*j/(inter)
#     numb = start_nb + (end_nb - start_nb)*j/(inter)
    

#     df = df.append(pd.DataFrame([[day, growth, height, thick, lea_len, lea_wid, lea_num, gaehwa, chaka, tomato, numb]], columns=['날짜', '생장길이(cm)', '화방높이(cm)', '줄기굵기(mm)', '엽장(cm)', '엽폭(cm)', '엽수(개)', 
#                            '개화군(점)', '착과군(점)', '열매수(개)', '최종화방번호(번)']), ignore_index=True)



In [ ]:
new_live1.iloc[70:80,:].round(2)

,생장길이(cm),화방높이(cm),줄기굵기(mm),엽장(cm),엽폭(cm),엽수(개),개화군(점),착과군(점),열매수(개),최종화방번호(번)
Date,,,,,,,,,,
2020-02-15,115.00,235.00,10.38,247.50,220.00,5.25,13.92,13.03,47.75,14.50
2020-02-16,116.79,240.00,10.46,248.21,218.21,6.00,13.95,13.09,47.29,14.57
2020-02-17,118.57,245.00,10.55,248.93,216.43,6.75,13.98,13.14,46.82,14.64
2020-02-18,120.36,250.00,10.63,249.64,214.64,7.50,14.01,13.20,46.36,14.71
2020-02-19,122.14,255.00,10.72,250.36,212.86,8.25,14.04,13.26,45.89,14.79
2020-02-20,123.93,260.00,10.80,251.07,211.07,9.00,14.07,13.32,45.43,14.86
2020-02-21,125.71,265.00,10.89,251.79,209.29,9.75,14.10,13.37,44.96,14.93
2020-02-22,127.50,270.00,10.97,252.50,207.50,10.50,14.13,13.43,44.50,15.00
2020-02-23,132.50,271.07,11.04,258.21,197.26,10.43,14.14,13.45,44.71,15.00


### 2-2-3. 제어 데이터

In [ ]:
ctrl1 = changedata(ctrl1)
ctrl2 = changedata(ctrl2)
ctrl3 = changedata(ctrl3)
ctrl4 = changedata(ctrl4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
ctrl2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 86 entries, 2021-02-01 to 2021-04-30
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   좌측일중천창 작동여부  86 non-null     float64
 1   우측일중천창 작동여부  86 non-null     float64
 2   우측이중천창 작동여부  86 non-null     float64
 3   차광커튼작동상태     86 non-null     float64
dtypes: float64(4)
memory usage: 3.4 KB


In [ ]:
ctrl2

,좌측일중천창 작동여부,우측일중천창 작동여부,우측이중천창 작동여부,차광커튼작동상태
Date,,,,
2021-02-01,0.00,0.00,0.0,23.94
2021-02-02,0.00,1.80,0.0,27.64
2021-02-03,0.29,2.79,0.0,32.32
2021-02-04,1.44,2.05,0.0,34.83
2021-02-05,3.32,2.81,0.0,36.99
...,...,...,...,...
2021-04-26,20.43,23.15,0.0,56.10
2021-04-27,14.29,15.55,0.0,61.51
2021-04-28,11.80,18.64,0.0,50.31


# 3. 시각화

In [ ]:
fig = go.Figure()
# we subtract 1 to stocks price data to show performance
fig.add_trace(go.Scatter(x=new_live1.index, y=new_live1['화방높이(cm)'], mode='lines+markers', name='height'))
fig.add_trace(go.Scatter(x=new_live1.index, y=new_live1['줄기굵기(mm)'], mode='lines+markers', name='tick'))
fig.add_trace(go.Scatter(x=new_live1.index, y=new_live1['엽장(cm)'], mode='lines+markers', name='leaf_len'))
fig.add_trace(go.Scatter(x=new_live1.index, y=new_live1['엽폭(cm)'], mode='lines+markers', name='leaf_wide'))
fig.add_trace(go.Scatter(x=new_live1.index, y=new_live1['열매수(개)'], mode='lines+markers', name='tomato'))


fig.update_layout(
    title='Tomato (Jan 2018 - Jan 2020)',
    xaxis_title='Date', yaxis_title='lenght(cm)'
)

## 3-1. live

In [ ]:
print(live2.corr())

                 주차  생장길이(cm)  화방높이(cm)  줄기굵기(mm)    엽장(cm)    엽폭(cm)  \
주차         1.000000 -0.412521  0.050332  0.445992  0.739724  0.787115   
생장길이(cm)  -0.412521  1.000000  0.542720  0.160292 -0.085487 -0.220019   
화방높이(cm)   0.050332  0.542720  1.000000  0.706385  0.464763  0.370209   
줄기굵기(mm)   0.445992  0.160292  0.706385  1.000000  0.871426  0.797572   
엽장(cm)     0.739724 -0.085487  0.464763  0.871426  1.000000  0.974690   
엽폭(cm)     0.787115 -0.220019  0.370209  0.797572  0.974690  1.000000   
엽수(개)      0.156551  0.360195  0.378990  0.606898  0.595988  0.507825   
개화군(점)     0.225424  0.358743  0.702861  0.556031  0.437203  0.338765   
착과군(점)     0.826232 -0.237101  0.027184  0.139683  0.417512  0.448837   
열매수(개)     0.731368 -0.325822 -0.170435  0.023386  0.345404  0.380361   
최종화방번호(번)  0.733735  0.041079  0.456737  0.481204  0.605250  0.576442   

              엽수(개)    개화군(점)    착과군(점)    열매수(개)  최종화방번호(번)  
주차         0.156551  0.225424  0.826232  0.731368   0.733735

In [ ]:
fig = go.Figure()
# we subtract 1 to stocks price data to show performance
fig.add_trace(go.Scatter(x=env1.index, y=env1['내부습도(%)'], mode='lines+markers', name='sub1'))
fig.add_trace(go.Scatter(x=env1.index, y=env1['내부온도(도)'], mode='lines+markers', name='tem1'))
fig.add_trace(go.Scatter(x=env1.index, y=env1['일사량(W/m-2·s)'], mode='lines+markers', name='sun1'))

fig.add_trace(go.Scatter(x=env3.index, y=env3['내부습도(%)'], mode='lines+markers', name='sub3'))
fig.add_trace(go.Scatter(x=env3.index, y=env3['내부온도(도)'], mode='lines+markers', name='tem3'))
fig.add_trace(go.Scatter(x=env3.index, y=env3['일사량(W/m-2·s)'], mode='lines+markers', name='sun3'))

In [ ]:
fig = go.Figure()
# we subtract 1 to stocks price data to show performance
fig.add_trace(go.Scatter(x=env1.index, y=env1['내부온도(도)'], mode='lines+markers', name='tem1'))

fig.add_trace(go.Scatter(x=env2.index, y=env2['내부온도(도)'], mode='lines+markers', name='tem2'))

fig.add_trace(go.Scatter(x=env3.index, y=env3['내부온도(도)'], mode='lines+markers', name='tem3'))

fig.add_trace(go.Scatter(x=env4.index, y=env4['내부온도(도)'], mode='lines+markers', name='tem4'))

In [ ]:
fig = go.Figure()
# we subtract 1 to stocks price data to show performance
fig.add_trace(go.Scatter(x=live1.index, y=live1['열매수(개)'], mode='lines+markers', name='tomato1'))
fig.add_trace(go.Scatter(x=live2.index, y=live2['열매수(개)'], mode='lines+markers', name='tomato2'))
fig.add_trace(go.Scatter(x=live3.index, y=live3['열매수(개)'], mode='lines+markers', name='tomato3'))
fig.add_trace(go.Scatter(x=live4.index, y=live4['열매수(개)'], mode='lines+markers', name='tomato4'))

In [ ]:
new_live2.iloc[:10,:]

,생장길이(cm),화방높이(cm),줄기굵기(mm),엽장(cm),엽폭(cm),엽수(개),개화군(점),착과군(점),열매수(개),최종화방번호(번)
Date,,,,,,,,,,
2021-01-09,0.00,193.80,9.60,363.80,272.50,9.75,4.75,6.75,68.70,8.75
2021-01-10,3.42,197.35,9.68,367.22,273.88,9.81,4.62,6.81,74.03,8.69
2021-01-11,6.85,200.90,9.75,370.65,275.25,9.88,4.50,6.88,79.35,8.62
2021-01-12,10.27,204.45,9.82,374.08,276.62,9.94,4.38,6.94,84.68,8.56
2021-01-13,13.70,208.00,9.90,377.50,278.00,10.00,4.25,7.00,90.00,8.50
2021-01-14,13.63,198.61,9.75,375.21,276.21,10.29,4.29,7.18,91.54,8.64
2021-01-15,13.56,189.23,9.61,372.93,274.43,10.57,4.32,7.36,93.07,8.79
2021-01-16,13.49,179.84,9.46,370.64,272.64,10.86,4.36,7.54,94.61,8.93
2021-01-17,13.41,170.46,9.32,368.36,270.86,11.14,4.39,7.71,96.14,9.07


In [ ]:
live3.iloc[5:15,:]

,주차,생장길이(cm),화방높이(cm),줄기굵기(mm),엽장(cm),엽폭(cm),엽수(개),개화군(점),착과군(점),열매수(개),최종화방번호(번)
Date,,,,,,,,,,,
2020-01-11,15.0,162.5,537.5,3.17,177.5,110.0,2.00,10.02,9.44,6.50,11.00
2020-01-18,16.0,170.0,112.5,7.33,240.0,155.0,2.50,11.11,10.37,8.00,12.00
2020-01-28,17.0,255.0,180.0,9.53,390.0,260.0,19.50,12.32,11.08,8.00,13.00
2020-02-01,18.0,77.5,195.0,12.11,327.5,285.0,0.75,12.65,11.42,0.00,13.25
2020-02-08,19.0,177.5,302.5,12.53,342.5,290.0,3.00,13.02,12.35,8.50,13.75
2020-02-15,20.0,115.0,235.0,10.38,247.5,220.0,5.25,13.92,13.03,47.75,14.50
2020-02-22,21.0,127.5,270.0,10.97,252.5,207.5,10.50,14.13,13.43,44.50,15.00
2020-02-29,22.0,162.5,277.5,11.43,292.5,135.8,10.00,14.22,13.56,46.00,15.00
2020-03-07,23.0,255.0,305.0,11.79,332.5,215.0,11.50,14.91,14.12,56.00,15.50


In [ ]:
corr_df = live2.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
# corr_df

s = corr_df.unstack()
# s

df = pd.DataFrame(s[s < 1].sort_values(ascending=False), columns=['corr']) # s==1 은 자기자신이기 때문에 제외
df.style.background_gradient(cmap='viridis')

,,corr
엽폭(cm),엽장(cm),0.970000
엽장(cm),엽폭(cm),0.970000
착과군(점),열매수(개),0.920000
열매수(개),착과군(점),0.920000
줄기굵기(mm),엽장(cm),0.870000
엽장(cm),줄기굵기(mm),0.870000
착과군(점),최종화방번호(번),0.860000
최종화방번호(번),착과군(점),0.860000
주차,착과군(점),0.830000
착과군(점),주차,0.830000


## 3-2. env

In [ ]:
print(env2.corr())

              내부CO2(ppm)   내부습도(%)      강우감지  일사량(W/m-2·s)   외부온도(도)  \
내부CO2(ppm)      1.000000 -0.190374  0.106662     -0.422267 -0.741889   
내부습도(%)        -0.190374  1.000000  0.405624     -0.304710  0.344375   
강우감지            0.106662  0.405624  1.000000     -0.510946  0.053641   
일사량(W/m-2·s)   -0.422267 -0.304710 -0.510946      1.000000  0.466237   
외부온도(도)        -0.741889  0.344375  0.053641      0.466237  1.000000   
내부온도(도)        -0.732016  0.135669 -0.137231      0.575753  0.821146   
지온(도)          -0.759758  0.151117 -0.084324      0.447783  0.907718   
외부풍속(m/s)      -0.104698  0.054309  0.093670     -0.064576  0.001221   

               내부온도(도)     지온(도)  외부풍속(m/s)  
내부CO2(ppm)   -0.732016 -0.759758  -0.104698  
내부습도(%)       0.135669  0.151117   0.054309  
강우감지         -0.137231 -0.084324   0.093670  
일사량(W/m-2·s)  0.575753  0.447783  -0.064576  
외부온도(도)       0.821146  0.907718   0.001221  
내부온도(도)       1.000000  0.944606  -0.158054  
지온(도)         0.944606  1.000

In [ ]:
corr_df = env2.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
# corr_df

s = corr_df.unstack()
# s

df = pd.DataFrame(s[s < 1].sort_values(ascending=False), columns=['corr'])
df.style.background_gradient(cmap='viridis')

,,corr
내부온도(도),지온(도),0.940000
지온(도),내부온도(도),0.940000
외부온도(도),지온(도),0.910000
지온(도),외부온도(도),0.910000
내부온도(도),외부온도(도),0.820000
외부온도(도),내부온도(도),0.820000
내부온도(도),일사량(W/m-2·s),0.580000
일사량(W/m-2·s),내부온도(도),0.580000
외부온도(도),일사량(W/m-2·s),0.470000
일사량(W/m-2·s),외부온도(도),0.470000


## 3-3. ctrl

In [ ]:
print(ctrl2.corr())

             좌측일중천창 작동여부  우측일중천창 작동여부  좌측이중천창 작동여부  우측이중천창 작동여부  좌측일중측창 작동여부  \
좌측일중천창 작동여부     1.000000     0.924604     0.858802     0.841724     0.895373   
우측일중천창 작동여부     0.924604     1.000000     0.789814     0.907732     0.870756   
좌측이중천창 작동여부     0.858802     0.789814     1.000000     0.852091     0.818982   
우측이중천창 작동여부     0.841724     0.907732     0.852091     1.000000     0.833715   
좌측일중측창 작동여부     0.895373     0.870756     0.818982     0.833715     1.000000   
차광커튼작동상태        0.737078     0.739025     0.553559     0.588440     0.672576   

             차광커튼작동상태  
좌측일중천창 작동여부  0.737078  
우측일중천창 작동여부  0.739025  
좌측이중천창 작동여부  0.553559  
우측이중천창 작동여부  0.588440  
좌측일중측창 작동여부  0.672576  
차광커튼작동상태     1.000000  


In [ ]:
corr_df = ctrl2.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
# corr_df

s = corr_df.unstack()
# s

df = pd.DataFrame(s[s < 1].sort_values(ascending=False), columns=['corr'])
df.style.background_gradient(cmap='viridis')

In [ ]:
env2.columns

Index(['내부CO2(ppm)', '내부습도(%)', '강우감지', '일사량(W/m-2·s)', '외부온도(도)', '내부온도(도)',
       '지온(도)', '외부풍속(m/s)'],
      dtype='object')

In [ ]:
df_merge1 = pd.merge(new_live1, env1, left_index=True, right_index=True, how='outer')
df_merge2 = pd.merge(new_live2, env2, left_index=True, right_index=True, how='outer')
df_merge3 = pd.merge(new_live3, env3, left_index=True, right_index=True, how='outer')
df_merge4 = pd.merge(new_live4, env4, left_index=True, right_index=True, how='outer')

In [ ]:
df_merge4.corr()

,생장길이(cm),화방높이(cm),줄기굵기(mm),엽장(cm),엽폭(cm),엽수(개),개화군(점),착과군(점),열매수(개),최종화방번호(번),...,이슬점온도(도),토양EC(dS/m),내부습도(%),지습(%),강우감지,일사량(W/m-2·s),외부온도(도),내부온도(도),지온(도),외부풍속(m/s)
생장길이(cm),1.000000,-0.206334,-0.202677,-0.208951,-0.450332,-0.234841,0.336444,0.335990,0.058333,0.340937,...,0.017425,-0.232085,0.064920,0.316134,-0.220352,-0.120337,-0.121139,-0.043276,-0.055284,-0.077909
화방높이(cm),-0.206334,1.000000,0.673721,0.070864,0.404145,-0.082172,-0.117665,-0.068289,-0.010681,-0.142707,...,-0.259363,-0.016506,0.069288,-0.027542,0.087739,-0.117426,-0.194761,-0.071414,-0.015260,0.230263
줄기굵기(mm),-0.202677,0.673721,1.000000,0.432574,0.819357,0.319126,-0.374603,-0.330011,-0.231378,-0.387761,...,-0.508472,0.106888,0.093427,0.027941,0.054124,0.074127,-0.466646,-0.150726,-0.225071,0.189849
엽장(cm),-0.208951,0.070864,0.432574,1.000000,0.618191,0.427837,-0.733319,-0.737317,-0.284820,-0.725564,...,-0.152409,0.252784,0.383824,-0.550938,0.237376,0.152294,0.466566,-0.181474,-0.386516,-0.188512
엽폭(cm),-0.450332,0.404145,0.819357,0.618191,1.000000,0.388508,-0.578506,-0.543909,-0.427121,-0.582609,...,-0.429989,0.095047,0.011010,-0.264670,0.136984,0.201195,-0.287889,-0.192007,-0.275738,0.113645
엽수(개),-0.234841,-0.082172,0.319126,0.427837,0.388508,1.000000,-0.655527,-0.650644,-0.379620,-0.643812,...,-0.393955,0.053037,0.259262,-0.321593,-0.100067,0.308286,-0.396077,-0.436161,-0.619724,0.178552
개화군(점),0.336444,-0.117665,-0.374603,-0.733319,-0.578506,-0.655527,1.000000,0.996847,0.398602,0.999420,...,0.257033,-0.262153,-0.560466,0.869119,-0.234426,0.007580,-0.589515,0.427050,0.563883,-0.087116
착과군(점),0.335990,-0.068289,-0.330011,-0.737317,-0.543909,-0.650644,0.996847,1.000000,0.379156,0.995574,...,0.216532,-0.238207,-0.542757,0.847158,-0.227285,0.021969,-0.615839,0.406764,0.529099,-0.052422
열매수(개),0.058333,-0.010681,-0.231378,-0.284820,-0.427121,-0.379620,0.398602,0.379156,1.000000,0.394148,...,0.033549,0.348684,-0.170651,0.108731,0.129760,-0.133859,-0.105161,0.198288,0.159075,0.167246
최종화방번호(번),0.340937,-0.142707,-0.387761,-0.725564,-0.582609,-0.643812,0.999420,0.995574,0.394148,1.000000,...,0.255866,-0.260736,-0.562534,0.867104,-0.242960,0.021991,-0.579367,0.426082,0.557713,-0.094672


# Modeling

생육, 환경 데이터 df_merge1,df_merge2,df_merge3,df_merge4

In [ ]:
df_merge4.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 233 entries, 2021-07-29 to 2022-03-18
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   생장길이(cm)      105 non-null    float64
 1   화방높이(cm)      105 non-null    float64
 2   줄기굵기(mm)      105 non-null    float64
 3   엽장(cm)        105 non-null    float64
 4   엽폭(cm)        105 non-null    float64
 5   엽수(개)         105 non-null    float64
 6   개화군(점)        105 non-null    float64
 7   착과군(점)        105 non-null    float64
 8   열매수(개)        105 non-null    float64
 9   최종화방번호(번)     105 non-null    float64
 10  내부CO2(ppm)    167 non-null    float64
 11  이슬점온도(도)      171 non-null    float64
 12  토양EC(dS/m)    158 non-null    float64
 13  내부습도(%)       170 non-null    float64
 14  지습(%)         170 non-null    float64
 15  강우감지          158 non-null    float64
 16  일사량(W/m-2·s)  166 non-null    float64
 17  외부온도(도)       167 non-null    float64
 18  내부온도(도)    

In [ ]:
ndf_merge4 = df_merge4[df_merge4["열매수(개)"].isnull() == False ]
ndf_merge4 = ndf_merge4[ndf_merge4["강우감지"].isnull() == False ]

ndf_merge4.head()

,생장길이(cm),화방높이(cm),줄기굵기(mm),엽장(cm),엽폭(cm),엽수(개),개화군(점),착과군(점),열매수(개),최종화방번호(번),...,이슬점온도(도),토양EC(dS/m),내부습도(%),지습(%),강우감지,일사량(W/m-2·s),외부온도(도),내부온도(도),지온(도),외부풍속(m/s)
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-27,0.00,239.50,9.20,482.25,442.25,10.50,13.84,12.93,86.00,14.50,...,10.84,1.0,54.44,69.10,0.00,139.67,2.26,9.96,15.27,0.31
2021-11-28,28.93,237.79,9.31,481.46,431.43,10.57,13.93,12.99,87.11,14.57,...,11.54,1.0,55.63,69.10,0.00,125.18,2.44,10.04,15.45,0.17
2021-11-29,57.86,236.07,9.42,480.68,420.61,10.64,14.01,13.06,88.21,14.64,...,11.80,1.0,55.42,69.26,0.00,132.54,3.80,10.07,15.34,0.10
2021-11-30,86.79,234.36,9.53,479.89,409.79,10.71,14.10,13.12,89.32,14.71,...,11.54,1.0,63.78,68.55,0.48,5.23,5.32,8.40,13.70,2.13
2021-12-01,115.71,232.64,9.64,479.11,398.96,10.79,14.19,13.19,90.43,14.79,...,9.83,1.0,60.72,68.65,0.00,50.79,-0.02,8.30,13.48,2.20


In [ ]:
ndf_merge4.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 37 entries, 2021-11-27 to 2022-01-02
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   생장길이(cm)      37 non-null     float64
 1   화방높이(cm)      37 non-null     float64
 2   줄기굵기(mm)      37 non-null     float64
 3   엽장(cm)        37 non-null     float64
 4   엽폭(cm)        37 non-null     float64
 5   엽수(개)         37 non-null     float64
 6   개화군(점)        37 non-null     float64
 7   착과군(점)        37 non-null     float64
 8   열매수(개)        37 non-null     float64
 9   최종화방번호(번)     37 non-null     float64
 10  내부CO2(ppm)    37 non-null     float64
 11  이슬점온도(도)      37 non-null     float64
 12  토양EC(dS/m)    37 non-null     float64
 13  내부습도(%)       37 non-null     float64
 14  지습(%)         37 non-null     float64
 15  강우감지          37 non-null     float64
 16  일사량(W/m-2·s)  37 non-null     float64
 17  외부온도(도)       37 non-null     float64
 18  내부온도(도)     

In [ ]:
ndf_merge4.describe()

,생장길이(cm),화방높이(cm),줄기굵기(mm),엽장(cm),엽폭(cm),엽수(개),개화군(점),착과군(점),열매수(개),최종화방번호(번),...,이슬점온도(도),토양EC(dS/m),내부습도(%),지습(%),강우감지,일사량(W/m-2·s),외부온도(도),내부온도(도),지온(도),외부풍속(m/s)
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,...,37.000000,37.000000,37.00000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
mean,181.405405,225.371622,9.479189,452.801351,384.737838,11.064865,15.309730,14.541622,86.275676,15.931081,...,11.086216,0.982162,58.21000,70.651892,0.030000,92.150541,0.548378,9.265405,14.406757,0.824324
std,56.621135,35.061554,0.956011,19.346166,28.293893,0.724629,0.797041,1.003279,4.255255,0.800357,...,0.799494,0.030654,2.38464,0.973498,0.086538,33.945081,3.418415,0.525751,0.637561,0.826907
min,0.000000,138.750000,7.390000,419.000000,344.750000,10.000000,13.840000,12.930000,77.000000,14.500000,...,8.500000,0.880000,53.94000,68.550000,0.000000,5.230000,-8.370000,8.070000,12.840000,0.080000
25%,176.790000,214.820000,9.170000,436.680000,363.390000,10.570000,14.680000,13.660000,83.250000,15.290000,...,10.730000,0.980000,56.44000,70.140000,0.000000,65.290000,-0.860000,8.990000,13.910000,0.250000
50%,193.930000,233.210000,9.540000,451.680000,379.610000,10.860000,15.390000,14.540000,86.790000,16.000000,...,11.160000,1.000000,58.25000,70.700000,0.000000,105.230000,1.240000,9.380000,14.590000,0.540000
75%,209.640000,239.500000,9.860000,470.320000,398.960000,11.500000,15.970000,15.540000,89.320000,16.640000,...,11.730000,1.000000,60.04000,71.430000,0.000000,121.290000,3.100000,9.640000,14.800000,1.210000
max,252.500000,290.000000,11.540000,482.250000,442.500000,12.750000,16.590000,16.020000,93.750000,17.200000,...,12.030000,1.000000,63.78000,72.030000,0.480000,139.670000,5.320000,10.070000,15.450000,3.150000


In [ ]:
env1

,내부CO2(ppm),내부습도(%),강우감지,일사량(W/m-2·s),외부온도(도),내부온도(도),외부풍속(m/s)
Date,,,,,,,
2019-12-24,284.44,83.22,0.00,0.42,1.09,11.58,0.57
2019-12-25,411.51,84.80,0.00,66.41,2.18,14.10,0.86
2019-12-26,424.04,89.44,0.19,16.37,3.98,12.20,1.29
2019-12-27,367.24,83.28,0.00,108.61,0.81,14.44,0.70
2019-12-28,353.65,82.62,0.00,105.49,-0.15,14.30,0.39
2019-12-29,392.29,87.90,0.40,37.29,2.02,12.94,1.07
2019-12-30,432.46,90.14,0.11,29.97,4.92,12.91,0.96
2019-12-31,365.54,82.94,0.00,113.95,-3.85,13.97,1.46
2020-01-01,352.02,86.28,0.00,72.82,-2.16,13.34,0.49


In [ ]:
ctrl1

,좌측일중천창 작동여부,우측일중천창 작동여부,차광커튼작동상태
Date,,,
2019-12-24,0.00,0.00,12.89
2019-12-25,0.00,1.78,24.12
2019-12-26,0.00,0.00,25.46
2019-12-27,0.36,1.39,23.40
2019-12-28,0.44,2.44,22.46
2019-12-29,0.00,0.38,26.06
2019-12-30,1.98,0.96,25.88
2019-12-31,1.41,0.12,21.41
2020-01-01,1.28,0.00,20.04
